In [2]:
import pandas as pd
import numpy as np
import re
from langdetect import detect
from sklearn.model_selection import train_test_split
import cld3
import math
from dataset import *
from main import *

In [449]:
train_subset = train.iloc[:1000]

In [450]:
train_subset.to_csv('data/train_subset.csv', index=False)

In [5]:
train = pd.read_csv("data/csv/train.csv")
train.head()

,Artist,Song,Genre,Language,Lyrics,reliable_lang
0,belle sebastian,legal man,Pop,en,"L-O-V-E love, it's coming back, it's coming ba...",en
1,pete townshend,a friend is a friend,Rock,en,When eyes meet in silence\nA pact can be made\...,en
2,blossoms,love talk,Rock,en,my eyes align with you\nyou're on my side\nand...,en
3,bukka white,parchman farm blues,Folk,en,Judge gimme me life this morn'in\nDown on Parc...,en
4,pat benatar,i won't,Rock,en,I was there when you cried like a baby\nWhen y...,en


In [9]:
train[train['Genre'].apply(lambda x: x in ['R&B'])]['Lyrics'].values

array(["I met a girl on Christmas day\nShe stole my heart took my breath away\nWe were young and quite naive\nYoung enough we still believed\n\nThat love never dies\nIt just grows throughout our lifetime\nAnd girl you know it's true\nHere's a Christmas song for you, baby\nHey girl\nMerry Christmas girl, yeah\n\nAt this special time of year\nI am thankful that you're still here\nThere'll be mistletoes and color lights for the Christmas tree\nThey barely define what you mean to me\n\nBaby love never dies\nIt just grows throughout our lifetime\nAnd girl you know it's true\nHere's a Christmas song for you, baby\nI love you girl, yeah\n\nIn my heart I always knew\nThat I would spend my whole life with you\nSo may your holidays be merry and be raised\nAnd may your world be at peace tonight\n\nBaby love never dies\nIt just grows throughout our lifetime\nAnd girl you know it's true\nYou're like a light in my life shining through\nIf there was only one thing I could do\nI'd sing this Christmas 

In [315]:
data = pd.read_csv("data/train.csv")
data = data[data['Language']=='en']
data = data[data['Lyrics'].apply(lambda x: '\n' in x)]
data = data.drop_duplicates(subset=['Artist', 'Song', 'Genre'], ignore_index=True)
data = data[data['Lyrics'].apply(lambda x: '---------' not in x)]
data['Lyrics'] = data['Lyrics'].apply(lambda x: re.sub('\[.*\]', ' ', x))

In [316]:
def check_en(line):
    result = cld3.get_language(line)
    lang = result[0]
    reliable = result[2]
    if reliable and lang == 'en':
        return 'en'
    else:
        return 'other'

In [328]:
len(data)

192123

In [317]:
data['reliable_lang'] = data['Lyrics'].apply(check_en)
data = data[data['reliable_lang']=='en']

In [321]:
indices = np.arange(len(data))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

train_data = data.iloc[train_index].reset_index(drop=True)
dev_data = data.iloc[dev_index].reset_index(drop=True)
test_data = data.iloc[test_index].reset_index(drop=True)

In [322]:
train_data.to_csv('data/csv/train.csv', index=False)
dev_data.to_csv('data/csv/dev.csv', index=False)
test_data.to_csv('data/csv/test.csv', index=False)

In [294]:
data['Genre'].value_counts()

Rock          96376
Pop           70438
Metal         15910
Jazz          11701
Folk           6922
Indie          5592
R&B            1572
Electronic      447
Name: Genre, dtype: int64

In [288]:
df = pd.read_csv('data/csv/test.csv')

In [290]:
df = df[df['Genre']=='Pop']

In [280]:
dev_data['Genre'].value_counts()

Rock          8679
Pop           6313
Metal         1490
Jazz          1035
Folk           603
Indie          497
R&B            171
Electronic      36
Name: Genre, dtype: int64

In [184]:
jazz = data[data['Genre']=='Jazz'].reset_index(drop=True)
indices = np.arange(len(jazz))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

In [242]:
path = 'data/train/jazz_train.txt'
with open(path,'w', encoding='utf-8', errors='ignore') as f:
    for i in train_index:
#         f.write("ARTIST NAME: "+jazz['Artist'].iloc[i]+'\n')
#         f.write("GENRE: "+jazz['Genre'].iloc[i]+'\n')
        f.write(jazz['Lyrics'].iloc[i])
        f.write('\n')

In [116]:
for g in list(data['Genre'].value_counts().index):
    df = data[data['Genre']==g]
    artist = np.array(df['Artist']).reshape(-1,1)
    genre = np.array(df['Genre']).reshape(-1,1)
    lyrics = np.array(df['Lyrics']).reshape(-1,1)

    out = np.hstack((artist, genre, lyrics))
    file_path = 'data/'+g+'_train.npy'
    np.save(file_path, out)

In [105]:
artist = np.array(rnb['Artist']).reshape(-1,1)
genre = np.array(rnb['Genre']).reshape(-1,1)
lyrics = np.array(rnb['Lyrics']).reshape(-1,1)

rnb_out = np.hstack((artist, genre, lyrics))
np.save('data/rnb_train.npy', rnb_out)

In [93]:
detect(rnb['Lyrics'].iloc[0])

'en'

In [90]:
rnb['detect_lang'] = rnb['Lyrics'].apply(lambda x: detect(x[:50]))

/Users/haowei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data['Genre'].value_counts()

Rock          103721
Pop            83620
Metal          17207
Jazz           11760
Folk            7023
Indie           6033
R&B             1575
Electronic       462
Name: Genre, dtype: int64

In [442]:
rnb = create_ngram_model(NgramModel, 'data/csv/train.csv', 4,0.0000001,genre='R&B')

In [437]:
rnb6 = create_ngram_model(NgramModel, 'data/csv/train.csv', 6,0.0000001,genre='R&B')

In [443]:
rnb.gen_next_sent('I like')

'I will in my loaded came'

In [396]:
test_df = pd.read_csv('data/csv/test.csv')
train_df = pd.read_csv('data/csv/train.csv')

In [351]:
idx = random.randint(0,len(test_df)-1)
lyric = test_df.iloc[idx]['Lyrics']
# tot_pp += model.perplexity(lyric)

In [3]:
spec_dict = {"dropout": 0.5,
                 "num_lstm_layers": 2,
                 "bilstm_flag": True,
                 "word_bilstm_flag": False,
                 "use_artist": True,
                 "char_hidden_dim": 128,
                 "char_emb_dim": 50,
                 "char_model_type": "LSTM",
                 "word_emb_dim": 128,
                 "pre_train_word_embedding": None,
                 "feature_emb_dim": 128,
                 "final_hidden_dim": 512,
                 "iterations": 2000,
                 "print_every": 100,
                 "plot_every": 50}
ds = Dataset('./data/csv/train.csv', subset=['R&B'])
vocab_size = ds.tokenize_corpus(word_tokenize)
print("Successfully built dataset...")
print("index of new line character: ", ds.word2idx['\n'])
print("index of comma: ", ds.word2idx[','])
print("index of <UNK>: ", ds.word2idx['<UNK>'])

Successfully built dataset...
index of new line character:  9754
index of comma:  8035
index of <UNK>:  10688


In [ ]:
with open('saved_data/artist_set.p', "wb") as fp:
    pickle.dump(ds.artist_set, fp)
with open('saved_data/genre_set.p', 'wb') as fp:
    pickle.dump(ds.genre_set, fp)
with open('saved_data/word2idx.p', 'wb') as fp:
    pickle.dump(ds.word2idx, fp)
with open('saved_data/idx2word.p', 'wb') as fp:
    pickle.dump(ds.idx2word, fp)
with open('saved_data/phones2index.p', 'wb') as fp:
    pickle.dump(ds.phones2index, fp)
with open('saved_data/index2phones.p', 'wb') as fp:
    pickle.dump(ds.index2phones, fp)

In [ ]:
model = WordSequence(spec_dict, ds)
model, loss_list = train(model, ds, spec_dict, num_lines=2)
torch.save(model.state_dict(), 'model3.pt')
plt.plot(loss_list)
plt.show()
pred_lines = predict(model, ds)
print(pred_lines)

In [129]:
inp, target, artist, genre, next_line = ds.random_lyric_chunks(path = "data/csv/train.csv", subset=["R&B"], num_lines=4, if_train=False)

In [103]:
# inp, target, artist, genre, next_line = ds.random_lyric_chunks(path = "data/csv/train.csv", subset=["R&B"], num_lines=2, if_train=False)
inp = "Are you ready ? "
target = inp
input_list = gen_input(ds, inp, target, artist, genre, if_train=False)
res = batchify_sequence_labeling(input_list, False)
(word_seq_tensor, feature_seq_tensors, word_seq_lengths, word_seq_recover, char_seq_tensor, char_seq_lengths,
     char_seq_recover, ph_inputs, ph_seq_lengths, ph_seq_recover, target_seq_tensor, mask) = res
batch_size = word_seq_tensor.size(0)
seq_len = word_seq_tensor.size(1)
genre_input = feature_seq_tensors[1]
artist_input = feature_seq_tensors[0]
outs = model(word_seq_tensor, genre_input, artist_input, word_seq_lengths, char_seq_tensor, char_seq_lengths, char_seq_recover,
        ph_inputs, ph_seq_lengths, ph_seq_recover)
outs = outs.view(batch_size * seq_len, -1)
score = F.log_softmax(outs, 1)
_, pred = torch.max(score, 1)
pred = pred.view(batch_size, seq_len)

In [106]:
pred.size()

torch.Size([1, 4])

In [108]:
score.size()

torch.Size([4, 10689])

In [115]:
score = F.log_softmax(o1,1)
_, pred = torch.max(score, 1)

In [116]:
pred

tensor([10578])

In [100]:
loss_function = nn.NLLLoss(ignore_index=0)

In [101]:
pred

tensor([10578, 10578, 10578, 10578])

In [89]:
target_seq_tensor.view(4)[-1].view(1).size()

torch.Size([1])

In [114]:
target_seq_tensor

tensor([[5748, 6473, 7967,  788]])

In [113]:
loss_function(score, target_seq_tensor.view(4)[-1].view(1))

tensor(4.7541, grad_fn=<NllLossBackward>)

In [63]:
target_seq_tensor.view(4)[-1]

tensor(788)

In [60]:
decode_lyrics(ds, target_seq_tensor.view(4))

'are you ready ? '

In [53]:
ds.all_characters.index('\n')

96

In [46]:
_, idx = torch.max(score, 1)

In [48]:
max(score[0])

tensor(-2.3579, grad_fn=<UnbindBackward>)

In [50]:
score[0][score[0] > -2.4]

tensor([-2.3579], grad_fn=<IndexBackward>)

In [47]:
idx

tensor([10578, 10578, 10578, 10578])

In [45]:
decode_lyrics(ds, idx)

'scented unbeatable scented motivating '

In [34]:
decode_lyrics(ds, pred[0])

'\n \n \n \n '

In [9]:
pred_lines = predict(model, ds)
print(pred_lines)

Start predicting next line..
prediction input:  Our love is like water/angels 
 Pinned down and abused 

prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 3331, 3863, 9343, 4211, 5617, 2971]])
oh
full word
input len to lstm generation:  12
prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 8930, 3863, 9343, 7563, 5617, 2971, 1330]])
yeah
full word
input len to lstm generation:  13
prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 8930, 3863, 9343, 7563, 5617, 2971, 1330,
         9474]])
me
full word
input len to lstm generation:  14
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5617,  8930,  3863,  9343,  7563,  5617,
          2971, 10420,  9474,  7643]])
)
full word
input len to lstm generation:  15
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5617,  8930,  3863,  9343,  7563,  5617,
          2971, 10420,  9474,  7643,  9616]])
.
full word
input len to lstm generation:  16
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5

In [19]:
(word_seq_tensor, feature_seq_tensors, word_seq_lengths, word_seq_recover, char_seq_tensor, char_seq_lengths, char_seq_recover, phone_seq_tensor, phone_seq_lengths, phone_seq_recover, target_seq_tensor, mask) = res

In [20]:
lr = 0.001
model = WordSequence(spec_dict, ds)
optimizer = optim.Adam(model.parameters(), lr=lr)

build char sequence feature extractor: LSTM ...
build char sequence feature extractor: LSTM ...


In [12]:
word_seq_tensor

tensor([[7676, 2313, 3358, 2017, 8924, 7537, 7537, 7676, 2313, 3358, 2017, 8924,
         7537]])

In [13]:
char_seq_tensor.size()

torch.Size([13, 6])

In [21]:
model(word_seq_tensor, feature_seq_tensors[0], feature_seq_tensors[1], word_seq_lengths, char_seq_tensor, char_seq_lengths, char_seq_recover, phone_seq_tensor, phone_seq_lengths, phone_seq_recover)


IndexError: index out of range in self

In [30]:
s = "Said I'm sorry baby ,  said forgive me baby\nNow you ain't never seen me on one"
tokens = re.split("[ ]+", s)

In [31]:
tokens

['Said',
 "I'm",
 'sorry',
 'baby',
 ',',
 'said',
 'forgive',
 'me',
 'baby\nNow',
 'you',
 "ain't",
 'never',
 'seen',
 'me',
 'on',
 'one']